In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
model = EASE_R_Recommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2561 (20.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    # Optuna suggest params
    topK = trial.suggest_int("topK", 1, 1000)
    l2_norm = trial.suggest_float("l2_norm", 1, 1e7, log=True)
    normalize_matrix = False
    
    recommender = model(URM_train)
    recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "topK" : 24,
    "l2_norm" : 37.54323189430143,
    "normalize_matrix" : False
}
# {'topK': 22, 'l2_norm': 30.57223051536002}
# {'topK': 24, 'l2_norm': 37.54323189430143} best
study_name = "ease-r-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=10)

[I 2023-11-30 14:56:28,834] Using an existing study with name 'ease-r-study' instead of creating a new one.


EASE_R_Recommender: URM Detected 457 ( 3.6%) users with no interactions.
EASE_R_Recommender: URM Detected 333 ( 1.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 2.10 min
EvaluatorHoldout: Processed 10077 (100.0%) in 3.00 sec. Users per second: 3355


[I 2023-11-30 14:58:47,223] Trial 155 finished with value: 0.029149674490549973 and parameters: {'topK': 32, 'l2_norm': 21.90136310855922}. Best is trial 131 with value: 0.03004205329780373.


EASE_R_Recommender: URM Detected 457 ( 3.6%) users with no interactions.
EASE_R_Recommender: URM Detected 333 ( 1.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 1.94 min
EvaluatorHoldout: Processed 10077 (100.0%) in 3.08 sec. Users per second: 3270


[I 2023-11-30 15:00:56,017] Trial 156 finished with value: 0.029416225224501703 and parameters: {'topK': 51, 'l2_norm': 36.04687475595589}. Best is trial 131 with value: 0.03004205329780373.


EASE_R_Recommender: URM Detected 457 ( 3.6%) users with no interactions.
EASE_R_Recommender: URM Detected 333 ( 1.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 2.06 min
EvaluatorHoldout: Processed 10077 (100.0%) in 2.93 sec. Users per second: 3438


[I 2023-11-30 15:03:12,189] Trial 157 finished with value: 0.029052793490126266 and parameters: {'topK': 26, 'l2_norm': 19.579549378317864}. Best is trial 131 with value: 0.03004205329780373.


EASE_R_Recommender: URM Detected 457 ( 3.6%) users with no interactions.
EASE_R_Recommender: URM Detected 333 ( 1.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 1.89 min
EvaluatorHoldout: Processed 10077 (100.0%) in 3.30 sec. Users per second: 3049


[I 2023-11-30 15:05:19,314] Trial 158 finished with value: 0.029324534418312502 and parameters: {'topK': 74, 'l2_norm': 45.087986144310406}. Best is trial 131 with value: 0.03004205329780373.


EASE_R_Recommender: URM Detected 457 ( 3.6%) users with no interactions.
EASE_R_Recommender: URM Detected 333 ( 1.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 


In [14]:
study.best_params

{'topK': 24, 'l2_norm': 37.54323189430143}

In [15]:
final = model(URM_train_validation)
final.fit(**study.best_params)

EASE_R_Recommender: URM Detected 226 ( 1.8%) users with no interactions.
EASE_R_Recommender: URM Detected 124 ( 0.6%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 2.43 min


In [16]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2225 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10413 (100.0%) in 3.37 sec. Users per second: 3086


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.093278                 0.158106  0.132708  0.048562    0.081839   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.270498  0.147366  0.109553  0.513301      0.357693  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.823944          0.422931    0.823944       0.055496   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.346571                   0.998603             0.159453   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [ ]:
## MAP = 0.0485282